In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.preprocess import *
from utils.model import *

# Read data

In [2]:
y = read_target_train()

In [3]:
train_data = read_data('train')

In [4]:
test_data = read_data('test')

# Preprocess data

In [5]:
X = preprocess_data(train_data)

In [6]:
X.head()

,NPLV,VES,T,SI,MN,S,P,CR,NI,CU,...,POL_amin,POL_amax,VDSYP_mean,VDSYP_sum,VDSYP_amin,VDSYP_amax,VSSYP_mean,VSSYP_sum,VSSYP_amin,VSSYP_amax
0,510008,263700.0,1396.0,0.44,0.22,0.023,0.097,0.03,0.01,0.03,...,0.77,7.07,365.250000,4383,171,442,1747.500000,20970,220,7300
1,510009,264500.0,1419.0,0.68,0.20,0.017,0.087,0.02,0.01,0.03,...,0.63,4.76,351.266667,5269,104,408,1585.333333,23780,10,9950
2,510010,263800.0,1384.0,0.56,0.26,0.017,0.096,0.03,0.01,0.03,...,0.65,4.08,345.153846,4487,104,442,1851.538462,24070,10,5050
3,510011,264000.0,1401.0,0.48,0.27,0.018,0.091,0.03,0.01,0.02,...,0.79,11.18,342.538462,4453,104,408,2100.000000,27300,320,5020
4,510012,263300.0,1422.0,0.47,0.23,0.018,0.096,0.02,0.01,0.03,...,0.67,5.00,347.812500,5565,104,442,1783.750000,28540,40,4980


# Train catboost

In [7]:
from catboost import CatBoostRegressor

In [7]:
model_tst = CatBoostRegressor(iterations = 100, verbose = 1000, random_state = config['random_state'])
model_C = CatBoostRegressor(iterations = 100, verbose = 1000, random_state = config['random_state'])

In [9]:
cross_val(X, y, model_tst, model_C)

Learning rate set to 0.245535
0:	learn: 27.8295749	total: 78.2ms	remaining: 7.74s
99:	learn: 9.2638233	total: 2.33s	remaining: 0us
Learning rate set to 0.245535
0:	learn: 0.0900959	total: 28.7ms	remaining: 2.84s
99:	learn: 0.0149930	total: 2.12s	remaining: 0us
Learning rate set to 0.245535
0:	learn: 28.4376078	total: 21.8ms	remaining: 2.16s


C:\Users\S\Documents\Education\Data Analysis\Competitions\evraz chugun\produv_o4ka\utils\model.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred['TST'] = model_tst.predict(test_data)
C:\Users\S\Documents\Education\Data Analysis\Competitions\evraz chugun\produv_o4ka\utils\model.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred['C'] = model_C.predict(test_data)


99:	learn: 9.9292115	total: 2.37s	remaining: 0us
Learning rate set to 0.245535
0:	learn: 0.0907682	total: 23.6ms	remaining: 2.34s
99:	learn: 0.0139789	total: 2.37s	remaining: 0us
Learning rate set to 0.245535
0:	learn: 28.5243557	total: 27.6ms	remaining: 2.74s
99:	learn: 9.5322244	total: 2.14s	remaining: 0us
Learning rate set to 0.245535
0:	learn: 0.0902107	total: 26.6ms	remaining: 2.64s
99:	learn: 0.0149355	total: 2.17s	remaining: 0us
Learning rate set to 0.245561
0:	learn: 28.2547745	total: 22.1ms	remaining: 2.19s
99:	learn: 9.7298569	total: 2.1s	remaining: 0us
Learning rate set to 0.245561
0:	learn: 0.0927052	total: 24ms	remaining: 2.37s
99:	learn: 0.0147913	total: 2.31s	remaining: 0us
Learning rate set to 0.245561
0:	learn: 28.0469339	total: 24.8ms	remaining: 2.45s
99:	learn: 9.5437482	total: 2.06s	remaining: 0us
Learning rate set to 0.245561
0:	learn: 0.0889498	total: 25ms	remaining: 2.48s
99:	learn: 0.0135623	total: 2.17s	remaining: 0us


[(0.9127272727272727, 0.6852300242130751),
 (0.9163636363636364, 0.6888619854721549),
 (0.9136363636363637, 0.6743341404358354),
 (0.9112658994548758, 0.6856796116504854),
 (0.920351302241066, 0.6638349514563107)]

In [10]:
model_tst.fit(X, y['TST'])

Learning rate set to 0.255495
0:	learn: 28.2097417	total: 68.2ms	remaining: 6.75s
99:	learn: 10.2479851	total: 2.37s	remaining: 0us


In [11]:
model_C.fit(X, y['C'])

Learning rate set to 0.255495
0:	learn: 0.0896764	total: 24.2ms	remaining: 2.4s
99:	learn: 0.0158407	total: 2.36s	remaining: 0us


In [12]:
save_catboost_models(model_tst, model_C)